In [1]:
import datetime
import time
from datetime import date

import pickle

import pandas as pd
import numpy as np

import os
import sys
import shutil

import matplotlib.pyplot as plt

In [2]:
from selenium import webdriver as wd
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver import ActionChains # scroll down 사용하기 위하여 선서

from bs4 import BeautifulSoup as bs
import requests

In [23]:
def collect_info(soup):
    condition = {}
    condition['회사명'] = soup.head.title.text[:-6]
    h2_sel = '#__next main div.title-form__group > h2'
    for h2 in soup.select(h2_sel):
#         print(h2.text)
        if h2.text == '모집조건':
            dl_dd = h2.parent.parent.next_sibling.select('dl')
            for dd in dl_dd:
#                 print("***", dd.dt.text, "---", dd.dd.text)
                condition[dd.dt.text] = dd.dd.text
        if h2.text == '근무지역':
            dl_dd = h2.parent.parent.parent.select('p')
#             print("ppp", dl_dd)
            for dd in dl_dd:
#                 print("***", '근무지역', "---", dd.text)
                condition['근무지역'] = dd.text
            
    return condition

In [4]:
service = Service(r"116/chromedriver.exe")
options = wd.ChromeOptions()
driver = wd.Chrome(service=service, options=options)

In [5]:
url = 'https://www.albamon.com/'
driver.get(url)
# driver.maximize_window()

In [6]:
q_sel = '#__next header form > div > div > div > input[type=text]'
q_element = driver.find_element(By.CSS_SELECTOR, q_sel) # 검색 창 element 선택

In [7]:
q_element.clear()
q_element.send_keys('시니어') # 검색어 입력
q_element.send_keys(Keys.ENTER)

In [8]:
# 검색 결과 갯수
num_s = driver.find_element(By.CSS_SELECTOR,'#__next  main div.list-header__item > span > strong').text.strip()
num_search = int(num_s)

# 페이지 갯수
num_page = divmod(num_search, 50)[0] + 1  # 몫에 1을 더함.

In [9]:
driver.find_element(By.ID, 'select-size').click() # 한 페이지에 최대 갯수 선택
driver.find_element(By.ID, 'option-button--3').click() # 최대 50개 선택

In [10]:
# button = "#__next  main button[type=button][aria-current]"  # 하단의 페이지 갯수 번호 최대 5개씩 표현됨.
# b_element = driver.find_elements(By.CSS_SELECTOR, button)

In [11]:
url_ = []
button = "#__next  main button[type=button][aria-current]" # 하단의 페이지 갯수 번호 최대 5개씩 표현됨.
b_element = driver.find_elements(By.CSS_SELECTOR, button)  # 페이지가 변할 때마다 지정을 해 줘야 함. 
for i in range(0, num_page):
    time.sleep(2)
    button = "#__next  main button[type=button][aria-current]" # 하단의 페이지 갯수 번호 최대 5개씩 표현됨.
    b_element = driver.find_elements(By.CSS_SELECTOR, button)  # 페이지가 변할 때마다 지정을 해 줘야 함. 
    b_element[i].click()
#     driver.refresh()
    time.sleep(2)
    
    url_sel = '#__next main ul.SimpleRecruitList_simple-recruit-list__ul__5KKhf li a[target=_self]'
    url_element = driver.find_elements(By.CSS_SELECTOR, url_sel) # https: 기 있는 모든 tag a target='_self' 선택
    
    # 한 페이지에 있는 최대 갯수 50개 url 선택
    for u in url_element:
        url_.append(u.get_attribute('href'))

In [12]:
len(url_)

140

In [32]:
column_sel = ['회사명', '모집마감', '모집인원', '모집분야', '근무지역', '성별', '학력', '연령', '우대사항']
df_con = pd.DataFrame(columns = column_sel)

for url in url_:
    r = requests.get(url)
    html = r.content
    soup = bs(html, 'html.parser')
    df_temp = pd.DataFrame([collect_info(soup)])
    df_con = pd.concat([df_con, df_temp])
    
df_con.set_index(np.arange(len(df_con.index)), inplace=True)

In [33]:
df_con.tail()

,회사명,모집마감,모집인원,모집분야,근무지역,성별,학력,연령,우대사항
135,[㈜고양시니어] | [행복한요양원 장기점] - (상담)사회복지사 채용,2023-11-26마감 45일전,1명,NaN,경기 김포시 전체,성별무관,"대학(2,3년제) 졸업 이상",연령무관,NaN
136,[㈜커넥트웰] | 콘텐츠기획/관리 직원 채용합니다.[유아-시니어콘텐츠],상시모집,0명(인원미정),NaN,경기 용인시 기흥구 전체,성별무관,대학(4년제) 졸업 이상,연령무관,NaN
137,[캉골키즈 롯데 중동점] | 캉골키즈 함께 일할 시니어 구인합니다.,2023-10-22마감 10일전,1명,NaN,경기 부천시 전체,성별무관,학력무관,연령무관,NaN
138,[㈜세스코] | (주)세스코 인천동부지사 시니어 인력(서비스 보조 SCA) 모집,2023-10-18마감 6일전,0명(인원미정),NaN,인천 연수구 전체,성별무관,학력무관,연령무관,NaN
139,[분당시니어] | [분당 행복한요양원(2관)] - 경력 사회복지사(사회복지팀장급) 채용,2023-10-18마감 6일전,0명(인원미정),NaN,경기 광주시 전체,성별무관,"대학(2,3년제) 졸업 이상",연령무관,NaN
